In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import subprocess
from subprocess import check_output
try:
    print(check_output(["ls", "../v2"]).decode("utf8"))
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
# Any results you write to the current directory are saved as output.


frame
video



In [2]:
import tensorflow as tf

In [3]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video
video_files = os.listdir("../v2/video/")

# Global variables
num_labels = 4716

In [4]:
def createTargetVec(labels):
    out = np.zeros((1, num_labels))
    for label in labels:
        out[0,label] = 1
    return out

In [5]:
def get_dataset(frames, samples=32, k=0):
    rgb_input = np.empty((samples, 100, 1024))
    audio_input = np.empty((samples, 100, 128))
    label_output = np.empty((samples, num_labels))
    sess = tf.InteractiveSession()
    for example in tf.python_io.tf_record_iterator(frames):        
        tf_seq_example = tf.train.SequenceExample.FromString(example)
        labels = tf_seq_example.context.feature['labels'].int64_list.value
        rgb_frame = np.zeros((100, 1024))
        audio_frame = np.zeros((100, 128))
        for i in range(100):
            rgb_frame[i] = tf.cast(tf.decode_raw(
                   tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                          ,tf.float32).eval()
            audio_frame[i] = tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval()
#         print("rgb_frame: ", rgb_frame)
        rgb_input[k] = rgb_frame
#         print("rgb_input: ", rgb_input)
        audio_input[k] = audio_frame
        label_output[k] = createTargetVec(labels[:])
        k += 1
        progress = (k / samples) * 100
        if int(progress) % 10 == 0:
            print("Progress", progress, "%")
        if k >= samples:
            break
    sess.close()
    return audio_input, rgb_input, label_output

In [6]:
# video_lvl_record = "../v2/video/train-1.tfrecord"
# frame_lvl_record = "../v2/frame/test30.tfrecord"

In [7]:
# audio_input, rgb_input, label_output = get_dataset(frame_lvl_record)

# X_train_rgb, X_test_rgb, _, _ = train_test_split(rgb_input, label_output, test_size=0.2, random_state=42)
# X_train_audio, X_test_audio, y_train, y_test = train_test_split(audio_input, label_output, test_size=0.2, random_state=42)

## Now were ready to create our model
****
** Since we're given sequential data, most likely I'll use an LSTM**

In [2]:
import keras
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.merge import dot, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import Input, Dense, Dropout, Bidirectional, Add, GlobalMaxPooling1D

Using TensorFlow backend.


### Frame Model
*** ***

In [10]:
x1 = Input(shape=(100,128),name='audio')
x2 = Input(shape=(100,1024),name='rgb_video')

In [11]:
fc_1_x1 = Dense(512, activation='tanh', name='fc_1_x1')(x1) 
fc_1_x2 = Dense(512, activation='tanh', name='fc_1_x2')(x2) 

In [12]:
# LSTM
lstm_1_x1 = LSTM(128, return_sequences=True, go_backwards=False, name='lstm_1_x1')(x1)
lstm_1_x2 = LSTM(1024, return_sequences=True, go_backwards=False, name='lstm_1_x2')(x2)

In [13]:
# Bidirectional_LSTM
lstm_2_x1 = LSTM(128, return_sequences=True, go_backwards=True, name='lstm_2_x1')(lstm_1_x1)
lstm_2_x2 = LSTM(1024, return_sequences=True, go_backwards=True, name='lstm_2_x2')(lstm_1_x2)

In [16]:
dropout_1_x1 = Dropout(rate=0.5, name="dropout_1_x1")(lstm_2_x1)
dropout_1_x2 = Dropout(rate=0.5, name="dropout_1_x2")(lstm_2_x2)

In [18]:
# fc_2_x1 = Dense(1, activation='softmax', name='fc_2_x1')(dropout_1_x1) 
# fc_2_x2 = Dense(1, activation='softmax', name='fc_2_x2')(dropout_1_x2) 

In [21]:
pool_1_x1 = GlobalMaxPooling1D(name='pool_1_x1')(fc_2_x1)
pool_1_x2 = GlobalMaxPooling1D(name='pool_1_x2')(fc_2_x2)

In [25]:
merge_1 = concatenate([pool_1_x1, pool_1_x2])

In [ ]:
fc_2 = Dense(1, activation='softmax', name='fc_2')(dropout_1_x1) 